In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive"

 532_project	   'Grocery_List & Chore Tracking.gsheet'   Spark_Word_count
 589		    lecture-04-parallelism.gdoc
'Colab Notebooks'   Resume_Vijayalakshmi.pdf


In [ ]:
!cd "/content/gdrive/My Drive/532_project"

In [ ]:
!ls "/content/gdrive/My Drive/532_project"

532_project.ipynb  SpeechReco.py  Transcription_experiments


In [ ]:
!pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 49.2 MB/s 


In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 51.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=8a1e022d9a63ac0fae92fcfb7395d817e10e4326fb4297ac86f507c6d5f04019
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
# PLEASE READ THE COMMENTS and PROVIDE REVIEW COMMENTS IN THE CODE
# This file take mp3 file as an input and produces <key,value> pair of <timestamp,transcribed text> in a single file using Google's Speech recognition library.
#This pair can later be pre-processed for deleting the articles, prepositions, conjections etc.. and can be added to the DB
#The granularity is currently set as 10 seconds. we can reduce or increase it depending.


#Trial Number#1: with 532 recording on Oct12.(1:15 hrs) <-- the transcribed text does not match with the actual words due to an unique accent.
#Trial Number#2: with 661 recording by Brian Levine(20 mins) <-- the transcribed output is perfect.

#TBD::: what is the time granularity? which timestamp style do we use? Need american accent dataset, to get proper result.
#Next step of code addition::: 
#       Loop through multiple videos
#       Timestamp correction
#       add the results in the DB
#       eliminate unimportant words

import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
import subprocess
import time


def mp3_to_wav_Conversion(mp3_src, wav_dst):
    subprocess.call(['ffmpeg', '-i', mp3_src,wav_dst])
    #test audio of the dst file
    test_audio = AudioSegment.from_file(wav_dst , "wav") 
    return test_audio

def split_files_with_timestamp(test_audio):
    chunk_length_ms = 10000 
    chunks = make_chunks(test_audio, chunk_length_ms) 
    return chunks

def writeInFile_key_value(filename, key, value):
    with open(filename, "a") as f:
        f.write(str(key)+"th second:\t"+value+"\n")

#Speech recognition object declaration
r = sr.Recognizer()

#Round 1 - converting Oct12.mp3 to oct12.wav --> Currently doing only for a single lecture to understand Speechrecognition
mp3_src = "/content/gdrive/My Drive/532_project/Transcription_experiments/661_recordings/Lec1.mp3"
wav_dst = "/content/gdrive/My Drive/532_project/output/Lec1.wav"
test_audio = mp3_to_wav_Conversion(mp3_src, wav_dst)

#split the huge audio file into smaller chunks of 1 sec each
chunks = split_files_with_timestamp(test_audio)
print(chunks)

#Export all of the individual chunks as wav files
for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    print("split#",i,"--",chunk_name)
    chunk.export("/content/gdrive/My Drive/532_project/output/Trial2_661_Lec1/"+chunk_name, format="wav") 

    #Round 1 - working on a random chunk439.wav to check the transcription recognize_google() function.
    #Round 2 - Doing for all the chunks.
    chunk_audioname  = "/content/gdrive/My Drive/532_project/output/Trial2_661_Lec1/"+chunk_name
    print(chunk_audioname)
    chunk_filename = "/content/gdrive/My Drive/532_project/output/Trial2_661_Lec1/TranscriptOutput.txt"
    with sr.AudioFile(chunk_audioname) as source:
        audio_listened = r.record(source)
        try:        
            tsc_value = r.recognize_google(audio_listened)
            print(tsc_value)
            #Here is where the timestamp is added.
            #Now, the granularity is set as 10 seconds and adding in a different file
            #Round 2 - So the key value is 0 , 10 , 20, 30 seconds. 
            #Round 3 - make it as timestamp and try adding in single file
            tsc_key = i*10
            writeInFile_key_value(chunk_filename, tsc_key, tsc_value)
        except sr.UnknownValueError as err:
                print("Empty", str(err))
        except sr.WaitTimeoutError as uErr:
                print("WaitTimeOutError",str(uErr))
                time.sleep(5)
                continue
        except sr.RequestError as rErr:
                time.sleep(5)
                continue

       
        
               


[<pydub.audio_segment.AudioSegment object at 0x7fe2d5571bd0>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571150>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571610>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571d90>, <pydub.audio_segment.AudioSegment object at 0x7fe2d55710d0>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571890>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571ad0>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571e10>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571250>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571d10>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571e50>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571490>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571f10>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571590>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571410>, <pydub.audio_segment.AudioSegment object at 0x7fe2d5571f90>, <pydub.audio_segment.Au

In [ ]:
#Preprocessing
import pandas as pd
from datetime import timedelta
from datetime import datetime

#read the text file as csv
read_file = pd.read_csv('/content/gdrive/My Drive/532_project/output/Trial2_661_Lec1/TranscriptOutput.txt',header=None,sep = ':')
#renaming columns
read_file.rename({0:'Seconds',1:'Transcribed_Text'},axis = 'columns',inplace=True)
#preprocessing column in dataframe
read_file['Seconds'] = read_file['Seconds'].apply(lambda x:x.replace('th','').replace('second',''))
#generating time stamp for the seconds
read_file['time_stamp'] = read_file['Seconds'].apply(lambda t:str(timedelta(seconds=int(t))))
#preprocessing time stamp column
read_file['time_stamp'] = read_file['time_stamp'].apply(lambda t:t.replace('0 days ',''))
# creating new columns - course name, course number and date
read_file['course_number'] = 661 
read_file['course_name'] = 'Secured Distributed Systems' 
read_file['Date'] = datetime.today()
#drop unncessary columns 
read_file.drop('Seconds',axis='columns',inplace=True)
read_file['Transcribed_Text'] = read_file['Transcribed_Text'].apply(lambda t:t.replace('\t',''))

In [ ]:
read_file

,Transcribed_Text,time_stamp,course_number,course_name,Date
0,I'm Brian Levine in this is a video for security,0:00:20,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
1,it systems in this video will talk about recov...,0:00:30,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
2,17 failures as the worst type of failure at th...,0:00:40,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
3,you're probably wondering what if anything's t...,0:00:50,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
4,idea that we want to offer a service to the wo...,0:01:00,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
...,...,...,...,...,...
108,strictly greater than a two-thirds majority of...,0:18:20,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
109,50% of the notes have to be non faulty for a c...,0:18:30,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
110,call until the answer is known where is blockc...,0:18:40,661,Secured Distributed Systems,2022-11-19 18:58:23.730043
111,not since they reach only a probabilistic or e...,0:18:50,661,Secured Distributed Systems,2022-11-19 18:58:23.730043


In [174]:
#converting pandas queries to pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
df_text = spark.read.text('/content/gdrive/My Drive/532_project/output/Trial2_661_Lec1/TranscriptOutput.txt')
# df2.show()
df_text = df_text.withColumn('Seconds', pyspark.sql.functions.split(df_text['value'], ':').getItem(0)) \
       .withColumn('Transcribed_Text',  pyspark.sql.functions.split(df_text['value'], ':').getItem(1)) 
df_text = df_text.select([column for column in df_text.columns if column not in ['value']])

df_text = df_text.withColumn('Seconds_formatted', regexp_replace('Seconds', ' second', ''))
df_text = df_text.withColumn('Seconds_formatted', regexp_replace('Seconds_formatted', 'th', ''))

df_text = df_text.select([column for column in df_text.columns if column not in ['Seconds']])

df_text = df_text.withColumnRenamed("Seconds_formatted","Seconds")

df_text = df_text.withColumn("Hrs", lit(df_text['Seconds']/3600))
df_text = df_text.withColumn("Hrs", df_text.Hrs.cast('int'))
df_text = df_text.withColumn("Mins", lit(df_text['Seconds']/60))
df_text = df_text.withColumn("Mins", df_text.Mins.cast('int'))
df_text = df_text.withColumn("Secs", lit(df_text['Seconds']%60))
df_text = df_text.withColumn("Secs", df_text.Secs.cast('int'))
df_text = df_text.withColumn('Hrs', when(length(df_text['Hrs']) < 2 ,lpad(df_text['Hrs'],2,'0')).otherwise(df_text['Hrs']))
df_text = df_text.withColumn('Mins', when(length(df_text['Mins']) < 2 ,lpad(df_text['Mins'],2,'0')).otherwise(df_text['Mins']))
df_text = df_text.withColumn('Secs', when(length(df_text['Secs']) < 2 ,lpad(df_text['Secs'],2,'0')).otherwise(df_text['Secs']))
df_text = df_text.withColumn("Timestamp", concat(col("Hrs"), lit(":"), col("Mins"), lit(":"), col("Secs")))

df_text = df_text.withColumn('Coursename',lit('Secured Distributed Systems'))
df_text = df_text.withColumn('Coursenumber',lit('661'))

#date column to appear here

# df_text = df_text.withColumn('Transcibed_Text', regexp_replace('Transcibed_Text','', ''))


In [175]:
#imporovments - store course number here and will have a different table store course name
# add a column called keywords - and extract date and keyword column as a different table
df_text.show()

+--------------------+-------+---+----+----+---------+--------------------+------------+
|    Transcribed_Text|Seconds|Hrs|Mins|Secs|Timestamp|          Coursename|Coursenumber|
+--------------------+-------+---+----+----+---------+--------------------+------------+
|\tI'm Brian Levin...|     20| 00|  00|  20| 00:00:20|Secured Distribut...|         661|
|\tit systems in t...|     30| 00|  00|  30| 00:00:30|Secured Distribut...|         661|
|\t17 failures as ...|     40| 00|  00|  40| 00:00:40|Secured Distribut...|         661|
|\tyou're probably...|     50| 00|  00|  50| 00:00:50|Secured Distribut...|         661|
|\tidea that we wa...|     60| 00|  01|  00| 00:01:00|Secured Distribut...|         661|
|\tgo down for eve...|     70| 00|  01|  10| 00:01:10|Secured Distribut...|         661|
|\tfind the primar...|     80| 00|  01|  20| 00:01:20|Secured Distribut...|         661|
|\tacknowledges th...|     90| 00|  01|  30| 00:01:30|Secured Distribut...|         661|
|\tthe problem is ...

In [184]:
# constructing different type of queries
# query by text--------enter (course number or name)
# sample query to check if contains a string entered
df_text_fil = df_text.filter(df_text.Coursename.contains('Secured Distributed Systems'))
df_text_fil = df_text.filter(df_text.Coursenumber.contains('661'))
df_text_fil = df_text.filter(df_text.Transcribed_Text.contains('systems'))
df_text_fil.select('Timestamp').show()
# query by date and text-----enter (course number or name)
df_text_fil = df_text.filter(df_text.Coursename.contains('Secured Distributed Systems'))
df_text_fil = df_text.filter(df_text.Coursenumber.contains('661'))
df_text_fil = df_text.filter(df_text.Transcribed_Text.contains('systems'))
df_text_fil.select('Timestamp').show()
# df.filter(df.location.contains('google.com'))

+---------+
|Timestamp|
+---------+
| 00:00:30|
| 00:00:40|
| 00:00:50|
| 00:05:30|
| 00:14:10|
| 00:18:10|
| 00:18:20|
| 00:18:30|
| 00:18:40|
| 00:19:00|
+---------+

